# CNN Model and Nearest Neighbors


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image
from tensorflow.keras.applications import ResNet101
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import os

2024-12-05 19:09:22.931894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 19:09:23.066221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 19:09:23.122282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 19:09:23.140146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 19:09:23.219548: I tensorflow/core/platform/cpu_feature_guar

In [2]:

# Preparing the dataset
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Added validation split

# Training and validation sets
training_set = train_datagen.flow_from_directory('dataset',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='training')

validation_set = train_datagen.flow_from_directory('dataset',
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   subset='validation')


Found 2416 images belonging to 10 classes.
Found 599 images belonging to 10 classes.


In [3]:
# # Defining the CNN model
# cnn= Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(10, activation='softmax')
# ])

# # Compiling the model
# cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Training the model
# history = cnn.fit(training_set, validation_data=validation_set, epochs=15)


In [4]:
# Load ResNet101 model pre-trained on ImageNet, excluding the fully connected layers (include_top=False)
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers (feature extraction)
base_model.trainable = False

# Add new fully connected layers for classification
new_classifier = Sequential([
    GlobalAveragePooling2D(),  # Add a GlobalAveragePooling layer to reduce dimensions
    Dense(1000, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout for regularization
    Dense(10, activation='softmax')  # Output layer (adjust number of classes)
])

# Combine the ResNet101 base model and the new classifier
cnn_model = Sequential([
    base_model,  # Add ResNet101 as the base
    new_classifier  # Add the new classifier on top
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

import warnings
warnings.filterwarnings("ignore")

# Train the model on the new dataset (only the fully connected layers will be trained)
history = cnn_model.fit(training_set, validation_data=validation_set, epochs=15)

I0000 00:00:1733404167.002100    1129 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733404167.111716    1129 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733404167.111759    1129 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733404167.115602    1129 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733404167.115647    1129 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Epoch 1/15


I0000 00:00:1733404176.584527    1947 service.cc:146] XLA service 0x7f2e78013e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733404176.584555    1947 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-12-05 19:09:36.784464: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-05 19:09:37.797668: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-12-05 19:09:39.075541: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11053', 144 bytes spill stores, 144 bytes spill loads



 2/76 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.2266 - loss: 2.6336  

I0000 00:00:1733404182.716550    1947 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.3885 - loss: 1.9149

2024-12-05 19:10:01.020754: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3315', 236 bytes spill stores, 236 bytes spill loads

2024-12-05 19:10:05.849095: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3308', 184 bytes spill stores, 184 bytes spill loads

2024-12-05 19:10:05.925526: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3315', 244 bytes spill stores, 244 bytes spill loads



76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 344ms/step - accuracy: 0.3897 - loss: 1.9114 - val_accuracy: 0.4674 - val_loss: 1.4254
Epoch 2/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 167ms/step - accuracy: 0.6191 - loss: 1.2498 - val_accuracy: 0.5927 - val_loss: 1.2469
Epoch 3/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 173ms/step - accuracy: 0.6586 - loss: 1.1520 - val_accuracy: 0.6411 - val_loss: 1.1848
Epoch 4/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 14s 187ms/step - accuracy: 0.7124 - loss: 0.9947 - val_accuracy: 0.6578 - val_loss: 1.0996
Epoch 5/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 166ms/step - accuracy: 0.7175 - loss: 0.9340 - val_accuracy: 0.6377 - val_loss: 1.0337
Epoch 6/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 15s 201ms/step - accuracy: 0.7380 - loss: 0.8378 - val_accuracy: 0.6327 - val_loss: 1.0917
Epoch 7/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 13s 177ms/step - accuracy: 0.7305 - loss: 0.8619 - val_accuracy: 0.7078 - val_loss: 0.9330
Epoch 8/15
76/76 ━━━━━━━━━━━━━━━━━━━━ 14s 179ms/step - accuracy: 0.7498 - loss: 0.7873 - val_accuracy: 0.706

In [5]:
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# import numpy as np

# img = load_img('photos/plan.jpg', target_size=(224, 224))
# img_array = img_to_array(img) / 255.0  # Rescale to [0, 1]
# img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# # Perform prediction
# predictions = model.predict(img_array)
# predicted_class_index = np.argmax(predictions)  # Get the index of the highest probability
# class_labels = training_set.class_indices  # Get mapping of class labels to indices
# class_labels = {v: k for k, v in class_labels.items()}  # Reverse the dictionary

# # Get the class name
# predicted_class = class_labels[predicted_class_index]
# confidence = predictions[0][predicted_class_index]

# print(f"Predicted class: {predicted_class} with confidence: {confidence:.2f}")  


In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Function to extract features from ResNet101 base (without the new fully connected layers)
def extract_features_fixed(image_path, base_model):
    img = load_img(image_path, target_size=(224, 224))  # Load image
    img_array = img_to_array(img) / 255.0  # Preprocess the image (normalize)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    features = base_model.predict(img_array)  # Extract features from ResNet101 base
    return features.flatten()  # Flatten the features to 1D

# Function to create feature list for nearest neighbors
def create_feature_list(dataset_dir, base_model):
    features_list = []
    image_paths = []
    
    for subdir, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_path = os.path.join(subdir, file)
                features = extract_features_fixed(file_path, base_model)
                features_list.append(features)
                image_paths.append(file_path)
    
    return np.array(features_list), image_paths

# Create feature list for the whole dataset
features_list, image_paths = create_feature_list("dataset", base_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━

In [1]:
# Function to find the nearest images based on extracted features
def find_nearest_images(image_path, base_model, n_neighbors=10):
    # Extract features for the input image
    image_features = extract_features_fixed(image_path, base_model)
    
    # Using NearestNeighbors to find the nearest images
    neighbors = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    neighbors.fit(features_list)
    
    distances, indices = neighbors.kneighbors([image_features])
    
    # Print the nearest images and their distance
    print(f"Nearest images to {image_path}:")
    for i in range(n_neighbors):
        print(f"{image_paths[indices[0][i]]} - Distance: {distances[0][i]:.4f}")
    
    # Optionally, plot the nearest images
    plt.figure(figsize=(12, 6))
    for i in range(n_neighbors):
        img = load_img(image_paths[indices[0][i]], target_size=(224, 224))
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Distance: {distances[0][i]:.4f}")
    plt.show()

# Example usage: Finding nearest images
find_nearest_images('photos/tt.jpg', base_model, n_neighbors=10)

NameError: name 'base_model' is not defined